In [5]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from selenium.webdriver.chrome.options import Options

# Set up WebDriver (Ensure you have installed ChromeDriver)
chrome_options = Options()
chrome_options.add_argument("--disable-popup-blocking")  # Blocks popups
chrome_options.add_argument("--disable-notifications")  # Blocks notification popups
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])



In [7]:
def get_rns_urls(trust):

    # Launch WebDriver with options
    driver = webdriver.Chrome(options=chrome_options)

    url = f"https://www.lse.co.uk/rns/{trust}/?filter=holdings-in-company"

    driver.get(url)

    # Wait for cookie popup and accept it
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
        )
        cookie_button.click()
        # print("Accepted cookies.")
    except:
        # print("No cookie popup found or already accepted.")
        pass

    # Wait for table to load
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        html_content = driver.page_source  # Get fully rendered HTML
        soup = BeautifulSoup(html_content, "html.parser")
        # Select tbody
        tbody = soup.find("tbody")

        # Extract rows from tbody
        rows = tbody.find_all("tr") if tbody else []

        rns_urls = []

        # Loop through rows and extract cell data
        for row in rows:
            cells = row.find_all("td")
            cell_texts = [cell.text.strip() for cell in cells]
            for td in cells:
                a_tag = td.find("a", href=True)  # Find <a> inside <td>
                if a_tag:
                    rns_url = urljoin("https://www.lse.co.uk", a_tag["href"])  # Construct full URL
                    rns_urls.append(rns_url)
                    # print(rns_url)
            # print(cell_texts, rns_url)

            # Open the link in Selenium
            # driver.get(rns_url)

            # try:
            #     cookie_button = WebDriverWait(driver, 5).until(
            #         EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
            #     )
            #     cookie_button.click()
            #     # print("Accepted cookies.")
            # except:
            #     # print("No cookie popup found or already accepted.")
            #     pass

            # # Wait for investor popup and accept it
            # try:
            #     cookie_button = WebDriverWait(driver, 5).until(
            #         EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Private Investor')]"))
            #     )
            #     cookie_button.click()
            #     # print("Accepted private investor.")
            # except:
            #     # print("No private investor popup found or already accepted.")
            #     pass

            # try: 
            #     # Wait for page load
            #     WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            # except:
            #     print("RNS Page took too long to load.")
            # # # Check if "Saba" is in the page source
            # # page_source = driver.page_source
            # # if "Saba" in page_source:
            # #     print(f"Found 'Saba' in: {rns_url}")
            # #     break
        
        return rns_urls

    except:
        print("Table not found or page took too long to load.")

    driver.quit()

def check_rns_content(rns_url):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(rns_url)

    # Wait for cookie popup and accept it
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
        )
        cookie_button.click()
        # print("Accepted cookies.")
    except:
        # print("No cookie popup found or already accepted.")
        pass

    # Wait for investor popup and accept it
    try:
        cookie_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Private Investor')]"))
        )
        cookie_button.click()
        # print("Accepted private investor.")
    except:
        # print("No private investor popup found or already accepted.")
        pass

    try: 
        # Wait for page load
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except:
        print("RNS Page took too long to load.")

    # Check if "Saba" is in the page source
    page_source = driver.page_source
    if "Saba" in page_source:
        print(f"Found 'Saba' in: {rns_url}")
        driver.quit()

        return True
    driver.quit()

    return False


rns_urls = get_rns_urls("EWI")

for rns_url in rns_urls:
    if check_rns_content(rns_url):
        break


https://www.lse.co.uk/rns/EWI/holdings-in-company-n9va8lljpt4m3oz.html
Found 'Saba' in: https://www.lse.co.uk/rns/EWI/holdings-in-company-n9va8lljpt4m3oz.html


In [10]:
trusts = ["AAIF","AAS","ADIG","AEI","AGT","AGVI","AIE","AJOT","ALW","AMAT","ANII","ARR","ASL","ATR","ATT","ATY","AUSC","AWEM","BAF","BASC","BBH","BEMO","BERI","BGCG","BGEU","BGFD","BGS","BGUK","BIOG","BNKR","BPCP","BPCR","BRAI","BRFI","BRGE","BRIG","BRLA","BRSC","BRWM","BUT","CCJI","CGI","CGT","CHI","CHIB","CLC","CLDN","CMPG","CMPI","CRT","CTUK","CTY","CYN","DGN","DIG","DIVI","EAT","EDIN","ENRG","EOT","ESCT","FA17","FAS","FCIT","FCSS","FEML","FEV","FGEN","FGT","FJV","FSV","FVEN","GCL","GPM","GSCT","HAN","HANA","HET","HFEL","HGEN","HHI","HHV","HINT","HOT","HRI","HSL","HVPE","HYG","IAT","IEM","IGC","IGET","INV","IPU","JAGI","JAM","JCGI","JCH","JEDT","JEGI","JEMA","JEMI","JFJ","JGC","JGGI","JII","JMG","JUGI","JUSC","KPC","LTI","LWDB","LWI","MCT","MHN","MIGO","MINI","MIX","MMIT","MNKS","MNL","MNP","MRC","MRCH","MTE","MTU","MUT","MWY","MYI","NAIT","NAS","NAVF","NSI","OIG","OIT","ONWD","OOA","OSEC","OVCT","PAC","PCFT","PCGH","PCT","PGV","PHI","PNL","PSH","RCOI","RICA","RIII","RKW","RMMC","SAIN","SCF","SCP","SDP","SDV","SEC","SHRS","SIHL","SJG","SMT","SOI","SSON","SST","STS","SVM","TEM","TENT","THRG","TMPL","TRY","UEM","USA","VEIL","VNH","WINV","WWH"]
results = {}
for trust in trusts:
    rns_urls = get_rns_urls(trust)
    results[trust] = False
    for rns_url in rns_urls:
        if check_rns_content(rns_url):
            results[trust] = True
            break

Found 'Saba' in: https://www.lse.co.uk/rns/AUSC/holdings-in-company-yt6tmll6fh8cmmf.html
Found 'Saba' in: https://www.lse.co.uk/rns/BASC/holdings-in-company-rmoplw934yu0z7s.html
Found 'Saba' in: https://www.lse.co.uk/rns/BBH/holdings-in-company-yyt7f22f9ewskwa.html
Found 'Saba' in: https://www.lse.co.uk/rns/BERI/total-voting-rights-li3jbnnno45qbs4.html
Found 'Saba' in: https://www.lse.co.uk/rns/BGEU/holdings-in-company-8p3gqv5158g3lrz.html
Found 'Saba' in: https://www.lse.co.uk/rns/BRSC/holdings-in-company-d06zod7cale48yk.html
Found 'Saba' in: https://www.lse.co.uk/rns/BRWM/holdings-in-company-rswar36209s0s64.html
Found 'Saba' in: https://www.lse.co.uk/rns/CYN/holdings-in-company-2h703jz6qd3rzeq.html
Found 'Saba' in: https://www.lse.co.uk/rns/EOT/holdings-in-company-crkt3cagomryr8k.html
Found 'Saba' in: https://www.lse.co.uk/rns/ESCT/holdings-in-company-cxokkyyt9dtn3w7.html
Found 'Saba' in: https://www.lse.co.uk/rns/GCL/holdings-in-company-xuasnv2xhlfcwk2.html
Found 'Saba' in: https://

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.110)
Stacktrace:
#0 0x56c89272553a <unknown>
#1 0x56c892220f00 <unknown>
#2 0x56c8921f6b6f <unknown>
#3 0x56c89229d95d <unknown>
#4 0x56c8922b3945 <unknown>
#5 0x56c892295343 <unknown>
#6 0x56c89226278a <unknown>
#7 0x56c8922639de <unknown>
#8 0x56c8926ef2cb <unknown>
#9 0x56c8926f3242 <unknown>
#10 0x56c8926dc7ac <unknown>
#11 0x56c8926f3df7 <unknown>
#12 0x56c8926c0b2f <unknown>
#13 0x56c8927141a8 <unknown>
#14 0x56c892714370 <unknown>
#15 0x56c8927243b6 <unknown>
#16 0x7a274e49ca94 <unknown>
#17 0x7a274e529c3c <unknown>


In [11]:
results

{'AAIF': False,
 'AAS': False,
 'ADIG': False,
 'AEI': False,
 'AGT': False,
 'AGVI': False,
 'AIE': False,
 'AJOT': False,
 'ALW': False,
 'AMAT': False,
 'ANII': False,
 'ARR': False,
 'ASL': False,
 'ATR': False,
 'ATT': False,
 'ATY': False,
 'AUSC': True,
 'AWEM': False,
 'BAF': False,
 'BASC': True,
 'BBH': True,
 'BEMO': False,
 'BERI': True,
 'BGCG': False,
 'BGEU': True,
 'BGFD': False,
 'BGS': False,
 'BGUK': False,
 'BIOG': False,
 'BNKR': False,
 'BPCP': False,
 'BPCR': False,
 'BRAI': False,
 'BRFI': False,
 'BRGE': False,
 'BRIG': False,
 'BRLA': False,
 'BRSC': True,
 'BRWM': True,
 'BUT': False,
 'CCJI': False,
 'CGI': False,
 'CGT': False,
 'CHI': False,
 'CHIB': False,
 'CLC': False,
 'CLDN': False,
 'CMPG': False,
 'CMPI': False,
 'CRT': False,
 'CTUK': False,
 'CTY': False,
 'CYN': True,
 'DGN': False,
 'DIG': False,
 'DIVI': False,
 'EAT': False,
 'EDIN': False,
 'ENRG': False,
 'EOT': True,
 'ESCT': True,
 'FA17': False,
 'FAS': False,
 'FCIT': False,
 'FCSS': Fal